# Writing out a USGSCSM ISD from a PDS3 Mex HRSC image

In [1]:
import os
# Update with where your metakernels are located
os.environ['ALESPICEROOT'] = '/scratch/spice/'
from ale.drivers.mex_drivers import MexHrscPds3NaifSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm
import json
import ale

## Instantiating an ALE driver

ALE drivers are objects that define how to acquire common ISD keys from an input image format, in this case we are reading in a PDS3 image using NAIF SPICE kernels for exterior orientation data. If the driver utilizes NAIF SPICE kernels, it is implemented as a [context manager](https://docs.python.org/3/reference/datamodel.html#context-managers) and will furnish metakernels when entering the context (i.e. when entering the `with` block) and free the metakernels on exit. This maintains the integrity of spicelib's internal data structures. These driver objects are short-lived and are input to a formatter function that consumes the API to create a serializable file format. `ale.formatters` contains available formatter functions. 

The ALESPICEROOT environment variable is used to find metakernels for different missions. You can update the path in the notebook to point to your metakernels directory. The directory is required to follow the NAIF direcrtory setup.

ALE has a two step process for writing out an ISD: 1. Instantiate your driver (in this case `LroLrocPds3LabelNaifSpiceDriver`) within a context and 2. pass the driver object into a formatter (in this case, `to_usgscsm`).  

Requirements:
 * A PDS3 Mex HRSC image
 * NAIF metakernels installed
 * A conda environment with ALE installed into it usisng the `conda install` command or created using the environment.yml file at the base of ALE.

In [2]:
# change to desired PDS3 image path 
fileName = '/home/kdlee/h5270_0000_ir2.img'

# metakernels are furnsh-ed when entering the context (with block) with a driver instance
# most driver constructors simply accept an image path 
with MexHrscPds3NaifSpiceDriver(fileName) as driver:
    # pass driver instance into formatter function
    usgscsm_dict = to_usgscsm(driver)


### Write ISD to disk 

ALE's USGSCSM formatter function returns a JSON. 

USGSCSM requires the ISD to be colocated with the image file with a `.json` extension in place of the image extension.


In [3]:
usgscsm_dict.keys()

dict_keys(['radii', 'sensor_position', 'sun_position', 'sensor_orientation', 'detector_sample_summing', 'detector_line_summing', 'focal_length_model', 'detector_center', 'starting_detector_line', 'starting_detector_sample', 'focal2pixel_lines', 'focal2pixel_samples', 'optical_distortion', 'image_lines', 'image_samples', 'name_platform', 'name_sensor', 'reference_height', 'name_model', 'interpolation_method', 'line_scan_rate', 'starting_ephemeris_time', 'center_ephemeris_time', 't0_ephemeris', 'dt_ephemeris', 't0_quaternion', 'dt_quaternion'])